# Streaming Internal Events

Event streaming is intrisinc to workflows and very easy to implement, as you can see in this example code:

```python
class SpecialEvent(Event):
    pass

class OtherEvent(Event):
    pass

class MyWorkflow(Workflow):
    ...

wf = MyWorkflow(...)

handler = wf.run()

async for event in handler.stream_events():
    if isinstance(event, SpecialEvent):
        print("This is a special event, hurray!")
    else:
        print("Not a special event :(")
```

Beyond streaming user-defined events, workflows can also stream internal events, such as changes in the state of the current step, input and output events, modifications of the workflow state and variation in the content of internal queues.

In the following example, we will see how we can leverage internal events streaming to expose details about the current workflow execution - while the workflow is running!

## 1. Install needed dependencies

In [ ]:
! pip install llama-index-workflows llama-cloud-services llama-index-llms-openai

## 2. Define events, workflow state and resources

In order for our workflow to work, we will need three things:

- Events classes defining the flow
- A workflow state representation
- External resources to inject into the workflow when needed

We will build a workflow that takes a document as input, extracts its raw text content and returns a summary based on that text.

### 2.1 Events

In [1]:
from workflows.events import Event, StartEvent, StopEvent


class InputDocumentEvent(StartEvent):
    document_path: str
    summary_prompt: str


class ParsedDocumentEvent(Event):
    document_content: str


class SummaryEvent(StopEvent):
    document_summary: str

### 2.2 State

In [2]:
from pydantic import BaseModel


class WorkflowState(BaseModel):
    summary_prompt: str = ""

### 2.3 Resources

For resources, we will use LlamaParse as a document parser, so you will need to set a `LLAMA_CLOUD_API_KEY` in your environment. If you do not have a LlamaCloud API key, you can [get one here](https://cloud.llamaindex.ai).

Also, you will need an OpenAI API key to use GPT-5 as a document summarizer.

In [ ]:
import os

os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."
os.environ["OPENAI_API_KEY"] = "sk-..."

In [3]:
from llama_cloud_services import LlamaParse
from llama_cloud_services.parse import ResultType
from llama_index.llms.openai import OpenAIResponses


async def get_document_parser(*args, **kwargs) -> LlamaParse:
    # we will use LlamaParse in agentic mode
    return LlamaParse(
        parse_mode="parse_page_with_agent",
        model="openai-gpt-4-1-mini",
        high_res_ocr=True,
        adaptive_long_table=True,
        outlined_table_extraction=True,
        output_tables_as_HTML=True,
        result_type=ResultType.MD,
    )


async def get_llm_summary(*args, **kwargs) -> OpenAIResponses:
    return OpenAIResponses(model="gpt-5-mini")

## 3. Define the workflow

In [4]:
from workflows import Workflow, Context, step
from workflows.resource import Resource
from typing import Annotated


class SummaryWorkflow(Workflow):
    @step
    async def get_document_content(
        self,
        ev: InputDocumentEvent,
        ctx: Context[WorkflowState],
        document_parser: Annotated[LlamaParse, Resource(get_document_parser)],
    ) -> ParsedDocumentEvent:
        async with ctx.store.edit_state() as state:
            state.summary_prompt = ev.summary_prompt
        result = await document_parser.aparse(ev.document_path)
        content = []
        if isinstance(result, list):
            for r in result:
                content.extend((await r.aget_markdown_documents()))
        else:
            content.extend((await result.aget_markdown_documents()))
        text_content = ""
        for document in content:
            text_content += document.text + "\n\n---\n\n"
        ctx.write_event_to_stream(ParsedDocumentEvent(document_content=text_content))
        return ParsedDocumentEvent(document_content=text_content)

    @step
    async def summarize_document(
        self,
        ev: ParsedDocumentEvent,
        ctx: Context[WorkflowState],
        llm: Annotated[OpenAIResponses, Resource(get_llm_summary)],
    ) -> SummaryEvent:
        state = await ctx.store.get_state()
        summary_prompt = state.summary_prompt
        summary_res = await llm.acomplete(
            f"Please create a summary of the following document:\n\n'''\n{ev.document_content}\n'''\n\nFollowing these instructions: {summary_prompt}"
        )
        return SummaryEvent(document_summary=summary_res.text)

## 4. Stream Events

In order to stream the internal events, we will pass `expose_internal = True` to the `stream_events` method on the workflow handler.

In [ ]:
!curl https://arxiv.org/pdf/2506.05176 -L -o qwen3_embed_paper.pdf

In [5]:
from workflows.events import StepStateChanged

wf = SummaryWorkflow(timeout=600)
handler = wf.run(
    start_event=InputDocumentEvent(
        document_path="qwen3_embed_paper.pdf",
        summary_prompt="This is a paper, so you should summarize it while still maintaining a scientific tone and its core concepts and findings",
    )
)

async for event in handler.stream_events(expose_internal=True):
    if isinstance(event, StepStateChanged):
        print("Name of current step:", event.name)
        print("State of current step:", event.step_state.value)
        print("Input event for current step:", event.input_event_name)
        print(
            "Output event of current step:",
            event.output_event_name or "No output event yet",
        )
    elif isinstance(event, ParsedDocumentEvent):
        print("Document has been successfully parsed!")
    else:
        continue

result = await handler
print(result.document_summary)

Name of current step: get_document_content
State of current step: running
Input event for current step: InputDocumentEvent
Output event of current step: No output event yet
Started parsing the file under job_id 995119d9-d55f-4872-aa2a-bded8ab3daf9
..Document has been successfully parsed!
Name of current step: get_document_content
State of current step: not_running
Input event for current step: <class '__main__.InputDocumentEvent'>
Output event of current step: <class '__main__.ParsedDocumentEvent'>
Name of current step: summarize_document
State of current step: running
Input event for current step: ParsedDocumentEvent
Output event of current step: No output event yet
Name of current step: summarize_document
State of current step: not_running
Input event for current step: <class '__main__.ParsedDocumentEvent'>
Output event of current step: <class '__main__.SummaryEvent'>
Here is a concise scientific summary of the Qwen3 Embedding technical report.

Overview
- The authors introduce the Q